In [1]:
import pandas as pd
import yfinance as yf

# Data Download

In [2]:
appl_df = yf.download(tickers="AAPL", start="2025-01-01", end="2025-12-31")
appl_df.head(10)

/var/folders/14/ljpzpqp91g1dpqy4sw00h0vm0000gn/T/ipykernel_3961/318833593.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  appl_df = yf.download(tickers="AAPL", start="2025-01-01", end="2025-12-31")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2025-01-02,242.752090,247.978453,240.731232,247.809205,55740700
2025-01-03,242.264313,243.080613,240.800930,242.264313,40244100
2025-01-06,243.896912,246.216423,242.105013,243.210016,45045600
2025-01-07,241.119492,244.444450,240.263363,241.886014,40856000
2025-01-08,241.607285,242.612747,238.969222,240.830798,37628900
2025-01-10,235.783630,239.078725,231.950958,238.929391,61710900
2025-01-13,233.344650,233.613439,228.685728,232.478572,49630700
2025-01-14,232.229675,235.056885,231.423325,233.693058,39435300


In [3]:
appl_df.to_csv('../data/research/APPLE_Bronze_data.csv')

# Data Preprocessing and Feature Engineering 

In [4]:
appl_df.columns

MultiIndex([( 'Close', 'AAPL'),
            (  'High', 'AAPL'),
            (   'Low', 'AAPL'),
            (  'Open', 'AAPL'),
            ('Volume', 'AAPL')],
           names=['Price', 'Ticker'])

In [5]:
appl_df.columns = appl_df.columns.get_level_values(0)
appl_df.head(10)

Price,Close,High,Low,Open,Volume
Date,,,,,
2025-01-02,242.752090,247.978453,240.731232,247.809205,55740700
2025-01-03,242.264313,243.080613,240.800930,242.264313,40244100
2025-01-06,243.896912,246.216423,242.105013,243.210016,45045600
2025-01-07,241.119492,244.444450,240.263363,241.886014,40856000
2025-01-08,241.607285,242.612747,238.969222,240.830798,37628900
2025-01-10,235.783630,239.078725,231.950958,238.929391,61710900
2025-01-13,233.344650,233.613439,228.685728,232.478572,49630700
2025-01-14,232.229675,235.056885,231.423325,233.693058,39435300
2025-01-15,236.799011,237.884115,233.374497,233.583558,39832000


In [6]:
import numpy as np


def add_price_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Returns
    df["return"] = df["Close"].pct_change()
    df["log_return"] = np.log(df["Close"] / df["Close"].shift(1))

    return df


def add_lag_features(df: pd.DataFrame, lags=(1, 3)) -> pd.DataFrame:
    df = df.copy()

    for lag in lags:
        df[f"return_lag_{lag}"] = df["log_return"].shift(lag)
        df[f"Close_lag_{lag}"] = df["Close"].shift(lag)

    return df


def add_rolling_features(df: pd.DataFrame, windows=(2, 3)) -> pd.DataFrame:
    df = df.copy()

    for window in windows:
        df[f"rolling_mean_{window}"] = df["log_return"].rolling(window).mean()
        df[f"rolling_std_{window}"] = df["log_return"].rolling(window).std()

    return df


def add_moving_averages(df: pd.DataFrame, windows=(3, 10)) -> pd.DataFrame:
    df = df.copy()

    for window in windows:
        df[f"sma_{window}"] = df["Close"].rolling(window).mean()
        df[f"ema_{window}"] = df["Close"].ewm(span=window, adjust=False).mean()

    return df


def add_volume_features(df: pd.DataFrame, window=10) -> pd.DataFrame:
    df = df.copy()

    df["volume_change"] = df["Volume"].pct_change()
    df[f"volume_rolling_mean_{window}"] = df["Volume"].rolling(window).mean()

    return df


def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df["day_of_week"] = df.index.dayofweek
    df["week_of_year"] = df.index.isocalendar().week.astype(int)
    df["month"] = df.index.month
    df["is_month_end"] = df.index.is_month_end.astype(int)

    return df


#def add_target(df: pd.DataFrame, horizon=1) -> pd.DataFrame:
#    """
#    Regression target: next-day log return
#    """
#    df = df.copy()
#    df["target_return"] = df["log_return"].shift(-horizon)
#    return df


def build_feature_table(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex")

    df = df.sort_index()

    df = add_price_features(df)
    df = add_lag_features(df)
    df = add_rolling_features(df)
    df = add_moving_averages(df)
    df = add_volume_features(df)
    df = add_time_features(df)
    #df = add_target(df)

    df = df.drop(columns=["Open", "Volume", "High", "Low", ])
    # Drop rows with NaNs caused by lags/rolling
    df = df.dropna()

    return df

In [7]:
df_features = build_feature_table(appl_df)
print(df_features.head())

Price            Close    return  log_return  return_lag_1  Close_lag_1  \
Date                                                                      
2025-01-16  227.232285 -0.040400   -0.041239      0.019485   236.799011   
2025-01-17  228.944519  0.007535    0.007507     -0.041239   227.232285   
2025-01-21  221.637573 -0.031916   -0.032436      0.007507   228.944519   
2025-01-22  222.822235  0.005345    0.005331     -0.032436   221.637573   
2025-01-23  222.653000 -0.000760   -0.000760      0.005331   222.822235   

Price       return_lag_3  Close_lag_3  rolling_mean_2  rolling_std_2  \
Date                                                                   
2025-01-16     -0.010398   233.344650       -0.010877       0.042938   
2025-01-17     -0.004790   232.229675       -0.016866       0.034469   
2025-01-21      0.019485   236.799011       -0.012465       0.028244   
2025-01-22     -0.041239   227.232285       -0.013553       0.026705   
2025-01-23      0.007507   228.944519     

In [8]:
df_features.tail(10)

Price,Close,return,log_return,return_lag_1,Close_lag_1,return_lag_3,Close_lag_3,rolling_mean_2,rolling_std_2,rolling_mean_3,...,sma_3,ema_3,sma_10,ema_10,volume_change,volume_rolling_mean_10,day_of_week,week_of_year,month,is_month_end
Date,,,,,,,,,,,,,,,,,,,,,
2025-12-16,274.609985,0.001824,0.001822,-0.015098,274.109985,-0.002694,278.029999,-0.006638,0.011965,-0.004126,...,275.666656,275.399520,278.250998,277.095961,-0.253139,39907560.0,1,51,12,0
2025-12-17,271.839996,-0.010087,-0.010138,0.001822,274.609985,0.000899,278.279999,-0.004158,0.008457,-0.007805,...,273.519989,273.619758,277.019998,276.140331,0.331755,40567560.0,2,51,12,0
2025-12-18,272.190002,0.001288,0.001287,-0.010138,271.839996,-0.015098,274.109985,-0.004426,0.008079,-0.002343,...,272.879995,272.904880,276.168997,275.422089,0.029757,41331720.0,3,51,12,0
2025-12-19,273.670013,0.005437,0.005423,0.001287,272.190002,0.001822,274.609985,0.003355,0.002925,-0.001143,...,272.566671,273.287447,275.657999,275.103530,1.801279,51068340.0,4,51,12,0
2025-12-22,270.970001,-0.009866,-0.009915,0.005423,273.670013,-0.010138,271.839996,-0.002246,0.010845,-0.001069,...,272.276672,272.128724,274.965997,274.351979,-0.747139,50904340.0,0,52,12,0
2025-12-23,272.359985,0.005130,0.005117,-0.009915,270.970001,0.001287,272.190002,-0.002399,0.010629,0.000208,...,272.333333,272.244355,274.483997,273.989799,-0.189485,50649210.0,1,52,12,0
2025-12-24,273.809998,0.005324,0.005310,0.005117,272.359985,0.005423,273.670013,0.005213,0.000137,0.000170,...,272.379995,273.027176,273.986996,273.957108,-0.395770,49136440.0,2,52,12,0
2025-12-26,273.399994,-0.001497,-0.001499,0.005310,273.809998,-0.009915,270.970001,0.001906,0.004814,0.002976,...,273.189992,273.213585,273.523996,273.855814,0.201624,47963820.0,4,52,12,0
2025-12-29,273.760010,0.001317,0.001316,-0.001499,273.399994,0.005117,272.359985,-0.000091,0.001990,0.001709,...,273.656667,273.486797,273.071997,273.838395,0.101915,46382050.0,0,1,12,0


In [9]:
df_features.to_csv("../data/research/APPLE_Silver_data.csv")

In [10]:
df_features.columns

Index(['Close', 'return', 'log_return', 'return_lag_1', 'Close_lag_1',
       'return_lag_3', 'Close_lag_3', 'rolling_mean_2', 'rolling_std_2',
       'rolling_mean_3', 'rolling_std_3', 'sma_3', 'ema_3', 'sma_10', 'ema_10',
       'volume_change', 'volume_rolling_mean_10', 'day_of_week',
       'week_of_year', 'month', 'is_month_end'],
      dtype='object', name='Price')